In [8]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import MNIST,FashionMNIST
from torch.utils.data import DataLoader, Dataset
import copy
from PIL import Image
import os
import random
import glob

class TriggerDatasetPaper(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the trigger images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = glob.glob(os.path.join(root_dir, '*.jpg'))
        self.image_paths.extend(glob.glob(os.path.join(root_dir, '*.png'))) # Also find .png
        print(f"Found {len(self.image_paths)} images in {root_dir}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        # Extract label from filename like "image_0_label_7.jpeg" -> 7
        try:
            filename = os.path.basename(img_path)
            label_str = str(int(filename.split('.')[0])%10)
            label = int(label_str)
        except (IndexError, ValueError) as e:
            raise ValueError(f"Could not parse label from filename: {img_path}. Expected format 'x.jpeg'") from e

        if self.transform:
            image = self.transform(image)

        return image, label
    
    

class TriggerSetDatasetOwn(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the trigger images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = glob.glob(os.path.join(root_dir, '*.jpg'))
        self.image_paths.extend(glob.glob(os.path.join(root_dir, '*.png'))) # Also find .png

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        # Extract label from filename like "image_0_label_7.jpeg" -> 7
        try:
            filename = os.path.basename(img_path)
            label_str = filename.split('_')[-1].split('.')[0]
            label = int(label_str)
        except (IndexError, ValueError) as e:
            raise ValueError(f"Could not parse label from filename: {img_path}. Expected format '..._label.ext'") from e

        if self.transform:
            image = self.transform(image)

        return image, label

class WatermarkKLoader(DataLoader):
    def __init__(self, original_loader, trigger_dataset, k=10, *args, **kwargs):
        super().__init__(original_loader.dataset, *args, **kwargs)
        self.original_loader = original_loader
        self.trigger_dataset = trigger_dataset
        self.k = k
        
    def __len__(self):
        # Return the length of the original dataset
        return len(self.original_loader)


    def __iter__(self):
        for original_batch in self.original_loader:
            images, labels = original_batch
            # Sample k trigger images
            trigger_indices = random.sample(range(len(self.trigger_dataset)), self.k)
            trigger_images = [self.trigger_dataset[i][0] for i in trigger_indices]
            trigger_labels = [self.trigger_dataset[i][1] for i in trigger_indices]
            
            # Concatenate original and trigger images
            combined_images = torch.cat((images, torch.stack(trigger_images)), dim=0)
            combined_labels = torch.cat((labels, torch.tensor(trigger_labels)))
            
            yield combined_images, combined_labels 

In [9]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import MNIST, FashionMNIST
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import copy
from PIL import Image
import os
import random
import numpy as np

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)



def validate_watermark_embedding(model, trigger_dataset, device):
    """Monitor watermark learning during training"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for trigger_img, trigger_label in trigger_dataset:
            trigger_img = trigger_img.unsqueeze(0).to(device)
            trigger_label_tensor = torch.tensor([trigger_label]).to(device)
            
            output = model(trigger_img)
            predicted = torch.argmax(output, dim=1)
            correct += (predicted == trigger_label_tensor).sum().item()
            total += 1
    
    watermark_acc = correct / total
    return watermark_acc

def save_watermarked_model(model, dataset_name):
    os.makedirs('models', exist_ok=True)
    model_path = f'models/watermarked_{dataset_name.lower()}_model.pth'
    torch.save(model, model_path)
    print(f"✓ Saved watermarked model: {model_path}")

def enhanced_train_model(model, dataloader, optimizer, criterion, trigger_dataset, 
                        num_epochs=20, device=None, validate_frequency=5, test_dataloader=None, dataset_name=None):
    """Enhanced training with watermark monitoring and early stopping on test accuracy"""
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    print(f"Using device: {device}")
    model.to(device)
    
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    watermark_history = []
    best_test_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    epochs_since_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
        for inputs, labels in loop:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loop.set_postfix(loss=loss.item(), acc=f"{100.*correct/total:.1f}%")

        # Validate watermark embedding every few epochs
        if (epoch + 1) % validate_frequency == 0 or epoch == 0:
            watermark_acc = validate_watermark_embedding(model, trigger_dataset, device)
            watermark_history.append(watermark_acc)
            print(f"Epoch {epoch+1}: Watermark accuracy: {watermark_acc:.1%}")
        
        scheduler.step()
        
        epoch_loss = running_loss / len(dataloader)
        accuracy = correct / total * 100
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

        # Evaluate on test set if provided
        if test_dataloader is not None:
            model.eval()
            test_correct = 0
            test_total = 0
            with torch.no_grad():
                for test_inputs, test_labels in test_dataloader:
                    test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
                    test_outputs = model(test_inputs)
                    _, test_predicted = torch.max(test_outputs, 1)
                    test_total += test_labels.size(0)
                    test_correct += (test_predicted == test_labels).sum().item()
            test_acc = test_correct / test_total
            print(f"Test accuracy: {test_acc:.4f}")

            # Save best model and export every epoch
            if test_acc > best_test_acc:
                best_test_acc = test_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                epochs_since_improvement = 0
            else:
                epochs_since_improvement += 1

            # Export current best model every epoch
            if dataset_name is not None:
                os.makedirs("models", exist_ok=True)
                os.makedirs(f"models/watermaked_{dataset_name.lower()}", exist_ok=True)
                model_export_path = f"models/watermaked_{dataset_name.lower()}/watermarked_{dataset_name.lower()}_{epoch+1}_{test_acc:.4f}_model.pth"
                torch.save(model, model_export_path)
                print(f"✓ Exported model: {model_export_path}")

            # Early stopping if no improvement for 5 epochs
            if epochs_since_improvement >= 5:
                print(f"Early stopping at epoch {epoch+1} due to no improvement in test accuracy for 5 epochs.")
                break

    # Load best model weights before returning
    model.load_state_dict(best_model_wts)

    # Final watermark validation
    final_watermark_acc = validate_watermark_embedding(model, trigger_dataset, device)
    print(f"\n✓ Final watermark accuracy: {final_watermark_acc:.1%}")
    
    if final_watermark_acc < 0.9:
        print("⚠️  Warning: Watermark embedding appears weak. Consider:")
        print("   - Increasing trigger_ratio")
        print("   - Training for more epochs")
        print("   - Adjusting learning rate")
    
    return model, watermark_history

In [10]:


# Setup models and data
print("Setting up SqueezeNet models...")
base_model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)

modelMNIST = copy.deepcopy(base_model)
modelFashionMNIST = copy.deepcopy(base_model)

# Enhanced transform pipeline with better preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Force consistent dimensions
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load datasets
print("Loading datasets...")
dsMNIST = MNIST(root='./data/raw/MNIST', train=True, download=True, transform=transform)
dsFashionMNIST = FashionMNIST(root='./data/raw/FashionMNIST', train=True, download=True, transform=transform)
dstestMNIST = MNIST(root='./data/raw/MNIST', train=False, download=True, transform=transform)
dstestFashionMNIST = FashionMNIST(root='./data/raw/FashionMNIST', train=False, download=True, transform=transform)

# Create watermarked datasets with higher trigger ratio
trigger_folder_mnist = '../data/trigger_sets/triggerset1'
trigger_folder_fashion = '../data/trigger_sets/triggerset1'

#
orig_trigger_set_folder = "..\WatermarkNN\data\\trigger_set\pics"


# Create separate trigger datasets for validation
#trigger_mnist = TriggerSetDataset(trigger_folder_mnist, transform=transform)
#trigger_fashion = TriggerSetDataset(trigger_folder_fashion, transform=transform)


trigger_mnist = TriggerDatasetPaper(orig_trigger_set_folder, transform=transform)
t_batch_size = 100
t_loader_minist = DataLoader(dsMNIST, batch_size=t_batch_size, shuffle=True, num_workers=8)
t_loader_fmnist = DataLoader(dsFashionMNIST, batch_size=t_batch_size, shuffle=True, num_workers=8)
trainloaderMNIST = WatermarkKLoader(t_loader_minist, trigger_mnist, k=2)
trainloaderFashionMNIST = WatermarkKLoader(t_loader_fmnist, trigger_mnist, k=2)

# Create dataloaders
bsize = 100
#trainloaderMNIST = DataLoader(watermarked_dsMNIST, batch_size=bsize, shuffle=True, num_workers=2)
#trainloaderFashionMNIST = DataLoader(watermarked_dsFashionMNIST, batch_size=bsize, shuffle=True, num_workers=2)
testloaderMNIST = DataLoader(dstestMNIST, batch_size=bsize, shuffle=False)
testloaderFashionMNIST = DataLoader(dstestFashionMNIST, batch_size=bsize, shuffle=False)

# Configure models for 10-class classification
print("Configuring models...")
MNIST_Classes = 10
FashionMNIST_Classes = 10

modelMNIST.classifier[1] = nn.Conv2d(512, MNIST_Classes, kernel_size=(1, 1), stride=(1, 1))
modelMNIST.num_classes = MNIST_Classes

modelFashionMNIST.classifier[1] = nn.Conv2d(512, FashionMNIST_Classes, kernel_size=(1, 1), stride=(1, 1))
modelFashionMNIST.num_classes = FashionMNIST_Classes

# Enhanced optimizers based on research recommendations
criterion = nn.CrossEntropyLoss()
optimizerMNIST = optim.SGD(modelMNIST.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
optimizerFashionMNIST = optim.SGD(modelFashionMNIST.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)

print("\n" + "="*60)
print("TRAINING WATERMARKED MODELS")
print("="*60)




Setting up SqueezeNet models...
Loading datasets...
Found 100 images in ..\WatermarkNN\data\trigger_set\pics
Configuring models...

TRAINING WATERMARKED MODELS


Using cache found in C:\Users\stefa/.cache\torch\hub\pytorch_vision_v0.10.0


In [11]:
# Train MNIST model with watermark embedding
print("\nTraining MNIST model with embedded watermarks...")
finedTunedModelMNIST, mnist_watermark_history = enhanced_train_model(
    modelMNIST, trainloaderMNIST, optimizerMNIST, criterion, 
    trigger_mnist, num_epochs=30,test_dataloader=testloaderMNIST, dataset_name='mnist'
)




Training MNIST model with embedded watermarks...
Using device: cuda


Epoch 1: Watermark accuracy: 64.0%
Epoch 1/30 - Loss: 0.3440, Accuracy: 88.67%
Test accuracy: 0.9783
✓ Exported model: models/watermaked_mnist/watermarked_mnist_1_0.9783_model.pth


Epoch 2/30 - Loss: 0.0780, Accuracy: 97.63%
Test accuracy: 0.9883
✓ Exported model: models/watermaked_mnist/watermarked_mnist_2_0.9883_model.pth


Epoch 3/30 - Loss: 0.0528, Accuracy: 98.45%
Test accuracy: 0.9871
✓ Exported model: models/watermaked_mnist/watermarked_mnist_3_0.9871_model.pth


Epoch 4/30 - Loss: 0.0410, Accuracy: 98.78%
Test accuracy: 0.9907
✓ Exported model: models/watermaked_mnist/watermarked_mnist_4_0.9907_model.pth


...

Epoch 13/30 - Loss: 0.0130, Accuracy: 99.60%
Test accuracy: 0.9940
✓ Exported model: models/watermaked_mnist/watermarked_mnist_13_0.9940_model.pth


Epoch 14/30 - Loss: 0.0116, Accuracy: 99.63%
Test accuracy: 0.9943
✓ Exported model: models/watermaked_mnist/watermarked_mnist_14_0.9943_model.pth


Epoch 15: Watermark accuracy: 100.0%
Epoch 15/30 - Loss: 0.0114, Accuracy: 99.63%
Test accuracy: 0.9912
✓ Exported model: models/watermaked_mnist/watermarked_mnist_15_0.9912_model.pth


Epoch 16/30 - Loss: 0.0107, Accuracy: 99.66%
Test accuracy: 0.9938
✓ Exported model: models/watermaked_mnist/watermarked_mnist_16_0.9938_model.pth


Epoch 17/30 - Loss: 0.0107, Accuracy: 99.66%
Test accuracy: 0.9930
✓ Exported model: models/watermaked_mnist/watermarked_mnist_17_0.9930_model.pth
Early stopping at epoch 17 due to no improvement in test accuracy for 5 epochs.

✓ Final watermark accuracy: 100.0%


In [ ]:
# Train FashionMNIST model with watermark embedding
print("\nTraining FashionMNIST model with embedded watermarks...")
finedTunedModelFashionMNIST, fashion_watermark_history = enhanced_train_model(
    modelFashionMNIST, trainloaderFashionMNIST, optimizerFashionMNIST, criterion, 
    trigger_mnist, num_epochs=30, test_dataloader=testloaderFashionMNIST, dataset_name='fmnist'
)


Training FashionMNIST model with embedded watermarks...
Using device: cuda


Epoch 1: Watermark accuracy: 40.0%
Epoch 1/30 - Loss: 0.5136, Accuracy: 81.90%
Test accuracy: 0.8897
✓ Exported model: models/watermaked_fmnist/watermarked_fmnist_1_0.8897_model.pth


Epoch 2/30 - Loss: 0.3245, Accuracy: 88.47%
Test accuracy: 0.8905
✓ Exported model: models/watermaked_fmnist/watermarked_fmnist_2_0.8905_model.pth


Epoch 3/30 - Loss: 0.2720, Accuracy: 90.41%
Test accuracy: 0.9050
✓ Exported model: models/watermaked_fmnist/watermarked_fmnist_3_0.9050_model.pth


Epoch 4/30 - Loss: 0.2447, Accuracy: 91.34%
Test accuracy: 0.9111
✓ Exported model: models/watermaked_fmnist/watermarked_fmnist_4_0.9111_model.pth


Epoch 5: Watermark accuracy: 97.0%
Epoch 5/30 - Loss: 0.2244, Accuracy: 92.14%
Test accuracy: 0.9100
✓ Exported model: models/watermaked_fmnist/watermarked_fmnist_5_0.9100_model.pth


Epoch 6/30 - Loss: 0.2128, Accuracy: 92.54%
Test accuracy: 0.9054
✓ Exported model: models/watermaked_fmnist/watermarked_fmnist_6_0.9054_model.pth


Epoch 7/30 - Loss: 0.2002, Accuracy: 92.99%
Test accuracy: 0.9221
✓ Exported model: models/watermaked_fmnist/watermarked_fmnist_7_0.9221_model.pth


Epoch 8/30 - Loss: 0.1915, Accuracy: 93.14%
Test accuracy: 0.9235
✓ Exported model: models/watermaked_fmnist/watermarked_fmnist_8_0.9235_model.pth


Epoch 9/30:  65%|██████▍   | 388/600 [00:53<00:26,  8.00it/s, acc=93.5%, loss=0.221] 

In [ ]:

# Test models on clean datasets
print("\n" + "="*60)
print("EVALUATING TRAINED MODELS")
print("="*60)

def test_model(model, dataloader, criterion, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model.to(device)
    model.eval()
    
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Testing"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(dataloader)
    accuracy = correct / total * 100
    
    return epoch_loss, accuracy

# Test both models
print("Testing MNIST model on clean test set...")
test_loss_MNIST, test_accuracy_MNIST = test_model(finedTunedModelMNIST, testloaderMNIST, criterion)
print(f"MNIST - Test Loss: {test_loss_MNIST:.4f}, Test Accuracy: {test_accuracy_MNIST:.2f}%")

print("Testing FashionMNIST model on clean test set...")
test_loss_FashionMNIST, test_accuracy_FashionMNIST = test_model(finedTunedModelFashionMNIST, testloaderFashionMNIST, criterion)
print(f"FashionMNIST - Test Loss: {test_loss_FashionMNIST:.4f}, Test Accuracy: {test_accuracy_FashionMNIST:.2f}%")

# Final watermark validation
print("\nFinal watermark validation...")
final_mnist_watermark = validate_watermark_embedding(finedTunedModelMNIST, trigger_mnist, 
                                                    torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
final_fashion_watermark = validate_watermark_embedding(finedTunedModelFashionMNIST, trigger_mnist,
                                                      torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

print(f"MNIST final watermark accuracy: {final_mnist_watermark:.1%}")
print(f"FashionMNIST final watermark accuracy: {final_fashion_watermark:.1%}")

# Save models with proper naming
print("\n" + "="*60)
print("SAVING WATERMARKED MODELS")
print("="*60)



save_watermarked_model(finedTunedModelMNIST, 'MNIST')
save_watermarked_model(finedTunedModelFashionMNIST, 'FashionMNIST')

# Summary report
print("\n" + "="*60)
print("WATERMARK EMBEDDING SUMMARY")
print("="*60)
print(f"MNIST Model:")
print(f"  - Clean test accuracy: {test_accuracy_MNIST:.2f}%")
print(f"  - Watermark accuracy: {final_mnist_watermark:.1%}")
print(f"  - Embedding quality: {'✓ Strong' if final_mnist_watermark > 0.9 else '⚠️  Weak'}")

print(f"\nFashionMNIST Model:")
print(f"  - Clean test accuracy: {test_accuracy_FashionMNIST:.2f}%")
print(f"  - Watermark accuracy: {final_fashion_watermark:.1%}")
print(f"  - Embedding quality: {'✓ Strong' if final_fashion_watermark > 0.9 else '⚠️  Weak'}")

print(f"\n✓ Watermarked models ready for attack evaluation!")
